<a href="https://colab.research.google.com/github/rcleveng/notebooks/blob/main/Langchain_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple langchain example using Google's Generate AI API.

You'll need an API key from Generative AI Studio.


Fist we install dependencies to use langserv.

In [ ]:
!pip install langchain langchain-cli "langserve[all]" google.generativeai | grep -v 'already satisfied' && echo "Installed dependencies"


In [18]:
# import for using secrets in colab
from google.colab import userdata

# basic langchain imports
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import google_palm
from langchain.chat_models import ChatGooglePalm
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


# Import things for functions
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
# agents
from langchain.agents import initialize_agent


# Get the key from https://makersuite.google.com
# Add your PALM_KEY as a secret to colab.

google_api_key = userdata.get('PALM_KEY')

In [3]:
SIMPLE_TEMPLATE="""
You are a helpful bot.
Answer to the question to the best of your ability.
If you do not know an answer state so.

Question: {question}
"""

Run this to turn on debugging and verbose output on all chains

In [7]:
from langchain.globals import set_verbose
from langchain.globals import set_debug

set_verbose(True)
set_debug(True)

In [36]:
import numexpr as ne

prompt = ChatPromptTemplate.from_template(SIMPLE_TEMPLATE)
model = ChatGooglePalm(google_api_key=google_api_key)

class SearchInput(BaseModel):
    query: str = Field(description="should be used to search the internet for current events", default="")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Search the internet for current events."""
    print("executing search-tool")
    return "LangChain SEARCH AGENT"

class CalculatorTool(BaseModel):
    query: str = Field(description="should be used to calculate math", default="")

@tool("calculator-tool", args_schema=SearchInput, return_direct=True)
def calculate(query: str) -> str:
    """should be used to calculate math"""
    print("executing calculator-tool")
    return str(ne.evaluate(query).item())
    return "LangChain CALCULATOR AGENT"

tools = [calculate, search]

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True)

# initialize agent with tools
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=model,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

out1 = agent.invoke("why is the sky blue?")
out2 = agent.invoke("What is 10 * 10?")
out3 = agent.invoke("How much is it if you add 20 to that?")

print("*** OUTPUT: ", out1['output'])
print("*** OUTPUT: ", out2['output'])
print("*** OUTPUT: ", out3['output'])


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "why is the sky blue?",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "why is the sky blue?",
  "chat_history": [],
  "agent_scratchpad": [],
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatGooglePalm] Entering LLM run with input:
{
  "prompts": [
    "System: Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly le